트레이닝 되지 않은 AutoModel, AutoTokenizer 로 pretrained된 BERT 모델을 로드하고,

코사인유사도를 계산하여 코사인유사도가 가장 높은 답변을 제공합니다.

임베딩은 사전 계산되어 계산 속도를 개선합니다.

In [9]:
import pandas as pd

df = pd.read_csv('file/health_result_label.csv')
df.head()

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()

In [10]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

# BERT 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")
model = AutoModel.from_pretrained("beomi/KcBERT-base")

question_embeddings = torch.load('embeddings/question_embeddings.pth', weights_only=True)

# 입력 문장을 임베딩으로 변환
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # 임베딩 벡터 반환

# 입력 질문과 사전 임베딩 간의 코사인 유사도 계산
def find_most_similar_answer(input_question, question_embeddings, answers):
    input_embedding = get_embedding(input_question)  # 입력 질문 임베딩

    max_similarity = -1
    best_answer = None

    # 각 사전 계산된 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        similarity = F.cosine_similarity(input_embedding, question_embedding).item()

        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity

# 입력 질문 예시
input_question = "정수기 위치가 어디죠?"
answers = df['answer'].tolist()

# 유사도 기반 답변 찾기
answer, max_similarity = find_most_similar_answer(input_question, question_embeddings, answers)
print("챗봇 답변:", answer)
print("유사도:", max_similarity)


챗봇 답변: 위치 정보를 조회하겠습니다. 병원에서 50m 떨어져 있는 곳에 위치해 있습니다.
유사도: 0.8512148857116699


In [11]:
## test_questions 데이터프레임 읽기
test_questions = pd.read_csv('file/test_questions_with_labels.csv')

# 모든 질문을 리스트로 변환하여 input_questions에 저장
input_questions = test_questions['question'].tolist()

# 결과를 저장할 리스트 초기화
results = []

# 각 질문에 대해 챗봇 응답 호출
for input_question in input_questions:
    answer, max_similarity = find_most_similar_answer(input_question, question_embeddings, answers)
    
    results.append({
        'input_question': input_question,
        'answer': answer,
       'max_similarity': max_similarity
    })
    
# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과 출력
results_df[:30]

,input_question,answer,max_similarity
0,화상연고도 처방이 되나요?,저희 병원에서 사용되는 의약품은 아래층에 행복 약국에서 구매가 가능합니다.,0.713620
1,오늘 두시 삼십분에 예약했어요. 언제 들어가나요?,어서 오십시오. 성함과 연락처를 앞에 적어주시면 확인 후 접수 도와드리겠습니다.,0.842398
2,예약한 시간보다 조금 늦었어요. 다시 접수해야 하나요?,대기하시기 전에 접수증 작성 부탁드립니다.,0.857856
3,"피가 계속 나는데, 저 죽는 걸까요?",긴급상황이라면 응급실로 이동하셔서 진료 보실 수 있습니다. 응급실 건물로 가주시면 ...,0.830222
4,접수 재등록해도 되나요?,마감 정산이 필요하여 지금 수납해 주시면 원활한 처리가 가능합니다.,0.777209
5,접수 시간이 지났는데 혹시 대기순번이 넘어갔나요?,확인해 도와드리겠습니다. 합의된 사항인지 확인이 필요합니다.,0.818078
6,접수가 왜이렇게 오래 걸려요?,앞서 이용해 주시는 손님이 많았습니다. 조금만 양해 부탁드립니다. 죄송합니다.,0.826650
7,"다음에 들어가는 순번인데, 잠깐 화장실 다녀와도 되나요?",네 알겠습니다. 다녀오시면 앞에 보이시는 2번 진료실로 들어가 주십시오.,0.823663
8,접수하려고요.,현재 30분 정도 대기하셔야 합니다. 진료 접수 도와드리도록 하겠습니다.,0.782673
9,진료 접수 어떻게 하죠?,바로 도와드리겠습니다. 옆쪽 통로를 통해 접수창구로 가시면 됩니다.,0.888841


In [12]:
# 결과를 CSV 파일로 저장
# results_df.to_csv('result csv/chatbot_responses_auto.csv', index=False)